In [34]:
import numpy as np
import pandas as pd

In [35]:
path = '/data/mountain/'

In [36]:
train = pd.read_json(path+'train.json', typ = 'frame')
train.shape
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [4]:
train.tail()

,tags,id,plylst_title,songs,like_cnt,updt_date
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000
115070,"[노래추천, 팝송추천, 팝송, 팝송모음]",100389,FAVORITE POPSONG!!!,"[26008, 456354, 324105, 89871, 135272, 143548,...",17,2020-04-18 20:35:06.000


### 모델 입력을 위한 train set 변형

In [37]:
train_input = pd.DataFrame(train['songs'].explode())
train_input

,songs
0,525514
0,129701
0,383374
0,562083
0,297861
...,...
115070,111365
115070,51373
115070,640239
115070,13759


In [38]:
train_input['playlist_id'] = train_input.index.map(train['id'].to_dict())
train_input

,songs,playlist_id
0,525514,61281
0,129701,61281
0,383374,61281
0,562083,61281
0,297861,61281
...,...,...
115070,111365,100389
115070,51373,100389
115070,640239,100389
115070,13759,100389


In [39]:
# 열명칭 변경: songs -> song_id
train_input.rename(columns = {'songs':'song_id'}, inplace = True)

# 각 플레이리스트별로 몇번째 노래인지 순서 표기
train_input['order'] = train_input.groupby('playlist_id').cumcount()
train_input

,song_id,playlist_id,order
0,525514,61281,0
0,129701,61281,1
0,383374,61281,2
0,562083,61281,3
0,297861,61281,4
...,...,...,...
115070,111365,100389,195
115070,51373,100389,196
115070,640239,100389,197
115070,13759,100389,198


In [ ]:
# csv 파일로 저장
train_input.to_csv(path+'train_bert4rec_input.csv', index=False)

In [41]:
# 샘플 토이데이터 csv 파일로 저장
train_input[:10000].to_csv(path+'train_bert4rec_input_toy.csv', index=False)

In [ ]:
# # csv 파일 불러오기
# train_input = pd.read_csv(path+'train_bert4rec_input.csv')

### Validation set 변형

In [12]:
valid = pd.read_json(path+'val.json', typ = 'frame')
valid.shape
valid.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [18]:
valid_input = pd.DataFrame(valid['songs'].explode())
valid_input

,songs
0,373313
0,151080
0,275346
0,696876
0,165237
...,...
23014,193899
23014,398886
23014,234875
23014,243850


In [19]:
valid_input['playlist_id'] = valid_input.index.map(valid['id'].to_dict())
valid_input

,songs,playlist_id
0,373313,118598
0,151080,118598
0,275346,118598
0,696876,118598
0,165237,118598
...,...,...
23014,193899,65189
23014,398886,65189
23014,234875,65189
23014,243850,65189


In [20]:
# 열명칭 변경: songs -> song_id
valid_input.rename(columns = {'songs':'song_id'}, inplace = True)

# 각 플레이리스트별로 몇번째 노래인지 순서 표기
valid_input['order'] = valid_input.groupby('playlist_id').cumcount()
valid_input

,song_id,playlist_id,order
0,373313,118598,0
0,151080,118598,1
0,275346,118598,2
0,696876,118598,3
0,165237,118598,4
...,...,...,...
23014,193899,65189,15
23014,398886,65189,16
23014,234875,65189,17
23014,243850,65189,18


In [22]:
# valid_input에 있는 결측치 확인
valid_input[valid_input.song_id.isna()]

,song_id,playlist_id,order
1,NaN,131447,0
8,NaN,80810,0
9,NaN,142007,0
17,NaN,2380,0
20,NaN,149069,0
...,...,...,...
22991,NaN,32537,0
22992,NaN,32812,0
22996,NaN,86721,0
23002,NaN,56570,0


In [23]:
# csv 파일로 저장
valid_input.to_csv(path+'valid_bert4rec_input.csv', index=False)

In [ ]:
# # csv 파일 불러오기
# valid_input = pd.read_csv(path+'valid_bert4rec_input.csv')